In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import fastai
from fastai.vision import *

# Any results you write to the current directory are saved as output.

In [ ]:
print(fastai.__version__)

In [ ]:
PATH = Path('.')
!ls {PATH}

In [ ]:
df = pd.read_csv(PATH/'../input/train.csv'); df.head()

In [ ]:
df['has_cactus'].hist()

In [ ]:
bs = 128

In [ ]:
tfms = get_transforms(do_flip=True, flip_vert=True)

In [ ]:
data = (ImageList.from_csv(csv_name='train.csv', path=PATH/'../input', folder='train/train')
            .split_by_rand_pct()
            .label_from_df(cols='has_cactus')
            .transform(tfms)
            .add_test_folder('test/test')
            .databunch(bs=bs)
           .normalize(imagenet_stats))

In [ ]:
data.show_batch(rows=3, figsize=(7, 7))

In [ ]:
data.classes, data.c

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=accuracy, path=PATH)

## Training

In [ ]:
learn.fit_one_cycle(4)

### Interpretation

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
losses, idxs = interp.top_losses()
interp.plot_top_losses(9, figsize=(7, 8))

In [ ]:
interp.plot_confusion_matrix(figsize=(3, 3))

### unfreeze

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(1)

In [ ]:
learn.recorder.plot()

## Prediction

In [ ]:
probs, preds = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
preds.shape, probs.shape

In [ ]:
!ls {PATH}/../input/test/test | wc -l

In [ ]:
ilst = data.test_ds.x

In [ ]:
fnames = [item.name for item in ilst.items]; fnames[:10]

In [ ]:
test_df = pd.DataFrame({'id': fnames, 'has_cactus': probs[:, 1]}); test_df

In [ ]:
test_df.to_csv('submission.csv', index=None)

In [ ]:
!head submission.csv